# Direct Attack  

## Linear Flow Eq  

$$\max_{\{c_t\}^T_1} U = \sum^T_{t=1}\beta^{t-1}u(c_t)$$
  subject to:
$$\sum^T_{t=1} c_t + k_{T+1} = k_1 \\ c_t > 0 \\ k_t > 0$$  

If we know the func $u(c_t)$, the discount factor $\beta$, the initial endowment $k_1$ and periods $T$, we can solve this problem.  

Here, $u(c)$ is log func, $\beta = 0.9$, $k_1 = 100$ and $T = 10$.  

In [ ]:
import numpy as np
from scipy.optimize import minimize
beta = 0.9
k0 = 100
T = 10

def obj(c):
    return -np.sum([beta**t * np.log(c[t]) for t in range(T)])

def resource_constraitnt(c):
    return k0 - np.sum(c)
const = [{'type':'eq', 'fun': resource_constraitnt}]

# variable constraint
bounds = [(0, None)] * T

# initial point
c0 = np.full(T, k0 / T)

res = minimize(obj, constraints=const, bounds=bounds, x0=c0,method='SLSQP', tol='1e-20')

paths = []
for t, c_t in enumerate(res.x):
    print(f"Period {t + 1}: Consumption = {c_t:.4f}")
    paths.append(c_t)
np.sum(paths)

Period 1: Consumption = 15.3534
Period 2: Consumption = 13.8181
Period 3: Consumption = 12.4362
Period 4: Consumption = 11.1926
Period 5: Consumption = 10.0734
Period 6: Consumption = 9.0660
Period 7: Consumption = 8.1594
Period 8: Consumption = 7.3435
Period 9: Consumption = 6.6091
Period 10: Consumption = 5.9482


100.0

## Non-Linear Flow Eq  

$$\max_{\{c_t\}^T_1} U = \sum^T_{t=1}\beta^{t-1}u(c_t)$$
subject to:
$$k_{t+1} = f(k_t-c_t, \theta) \\ c_t > 0 \\ k_t > 0$$  

Here, the pf is Cobb-Douglas:  
$$k_{t+1} = f(k_t-c_t, \theta)= \theta(k_t - c_t)^\alpha$$  

In [127]:
beta = 0.9
T = 10
K1 = 100
theta = 1.2
alpha = 0.98

def flow_utility(CK_flat):
    CK = CK_flat.reshape(T, 2)
    c = CK[:, 0]
    t = np.arange(1, T+1)
    V = np.sum(beta**(t - 1) * np.log(c))
    return -V

def flow_constraint(CK_flat):
    CK = CK_flat.reshape(T, 2)
    c = CK[:, 0]
    cap = CK[:, 1]
    k = np.concatenate(([K1], cap))  # [K1, k2, ..., k_{T+1}]

    deq = []
    for t in range(T):
        usable = k[t] - c[t]
        if usable < 0:
            # 非現実的な入力なら強制的に大きな制約違反にする
            return np.ones(T) * 1e6
        deq.append(k[t+1] - (theta * usable**alpha))
    return np.array(deq)

constraints = [{
    'type': 'eq',
    'fun': flow_constraint
}]

guess_C = np.full(T, 5.0)  
guess_K = np.linspace(90, 20, T) 
guess = np.column_stack([guess_C, guess_K]).flatten()

lb = np.zeros((T, 2)).flatten()
ub = np.full((T, 2), 100.0).flatten()
bounds = [(lb[i], ub[i]) for i in range(2 * T)]

opt_result = minimize(
    fun=flow_utility,
    x0=guess,
    bounds=bounds,
    constraints=constraints,
    method='SLSQP',
    options={
        'ftol': 1e-10,
        'maxiter': 2000,
        'disp': True
    }
)

if opt_result.success:
    CK_opt = opt_result.x.reshape(T, 2)
    C_opt = CK_opt[:, 0]
    K_opt = CK_opt[:, 1]
    print("Optimal consumption and capital path:")
    for t in range(T):
        print(f"t={t+1}: c = {C_opt[t]:.4f}, k = {K_opt[t]:.4f}")
else:
    print("Optimization failed:", opt_result.message)


C:\Users\Tsunc\AppData\Local\Temp\ipykernel_4552\97905264.py:11: RuntimeWarning: divide by zero encountered in log
  V = np.sum(beta**(t - 1) * np.log(c))


Optimization terminated successfully    (Exit mode 0)
            Current function value: -17.658853447604137
            Iterations: 999
            Function evaluations: 25058
            Gradient evaluations: 998
Optimal consumption and capital path:
t=1: c = 16.5168, k = 91.6956
t=2: c = 15.8437, k = 83.4735
t=3: c = 15.4496, k = 75.0221
t=4: c = 15.0586, k = 66.2996
t=5: c = 14.8554, k = 57.0547
t=6: c = 14.3864, k = 47.4991
t=7: c = 14.1407, k = 37.3183
t=8: c = 14.1507, k = 26.1074
t=9: c = 13.8951, k = 13.9393
t=10: c = 13.9393, k = 0.0000


# Dynamic Programming  

$$V_{T+1}(k_{T+1}) = 0$$

$$V_T(k_T) = \max_{c_T\in(0,k_T]}\{u(c_T)+\beta V_{T+1}(k_{T+1}) \}$$


In [ ]:
def backwardinduction(beta, period, initial_endowment, steps):
    
    k_list = np.arange(0,initial_endowment+1,steps)
    n_k = len(k_list)
    
    V = np.hstack([np.full((n_k, period), np.nan), np.zeros((n_k, 1))])
    aux = np.full((n_k, n_k, period), -np.inf)
    
    for t in range(period-1,-1,-1):
        for ink in range(n_k):
            for outk in range(ink+1):
                c = k_list[ink] - k_list[outk]
                if c > 0:
                    aux[ink,outk,t] = np.log(c) + beta * V[outk, t+1]
        V[:,t] = np.max(aux[:,:,t],axis=1)
    
    vf = np.full(period,np.nan)
    kap = np.full(period+1, np.nan)
    kap[0] = initial_endowment
    con = np.full(period,np.nan)
    
    for t in range(period):
        k_index = np.where(k_list==kap[t])[0][0]
        vf[t] = V[k_index, t]
        outk_index = np.argmax(aux[k_index, :, t])
        kap[t + 1] = k_list[outk_index]
        con[t] = kap[t] - kap[t+1]
    res = np.column_stack([kap[:-1], con])
    return res

b = 0.9
T = 10
k0 = 100.0
grid = 0.25
backwardinduction(beta=b, period=T, initial_endowment=k0, steps=grid)

array([[100.  ,  15.35],
       [ 84.65,  13.8 ],
       [ 70.85,  12.45],
       [ 58.4 ,  11.2 ],
       [ 47.2 ,  10.1 ],
       [ 37.1 ,   9.05],
       [ 28.05,   8.15],
       [ 19.9 ,   7.35],
       [ 12.55,   6.6 ],
       [  5.95,   5.95]])

### introduce stochastic term  

The stochastic model does not have a single optimal solution

In [203]:
e = np.array([-2,2])
prob = np.array([0.5,0.5])
beta = 0.9
theta = 1.2
alpha = 0.98
k0 = 100.0
grid = 0.1
T = 10

def finite_stochastic(beta, initial_endowment, epsilon, pi, grid, period, theta, alpha):
    
    k_list = np.arange(0,k0+max(e),grid)
    n_k = len(k_list)
    
    V = np.hstack([np.full((n_k,period), np.nan), np.zeros((n_k, 1))])
    aux = np.full((n_k, n_k, period), -np.inf)
    
    for t in range(period-1,-1,-1):
        for ink in range(n_k):
            for outk in range(ink+1):
                c = k_list[ink] - (k_list[outk]/theta) ^ (1/alpha)
                nextkl = theta*(k_list[ink] - c) ^ alpha + e[0]
                nextkl[nextkl < 0] = 0
                nextkh = theta*(k_list[ink] - c) ^ alpha + e[1]
                
                lindex = np.round(nextkl / grid).astype(int)
                hindex = np.round(nextkh / grid).astype(int)
                nextV = pi[0] * V[lindex,t+1] + pi[1] * V[hindex,t+1]
                
                aux[ink,outk,t] = np.log(c) + beta * nextV
        V[:,t] = np.max(aux[:,:,t],axis=1)


# Estimating Finite Horizon Models  

So far, it is assumed that we know the structural params, $\beta$, $\alpha$ and $\theta$.  
Here, we don't know them, and try to estimate them.  